In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn.functional import one_hot
import torch_geometric as pyg
import networkx as nx
import torch.nn.functional as F
from torch_geometric.datasets import ZINC
from torch_geometric.utils import to_networkx
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt

In [2]:
from gcn_model import GCNNet, GCN

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
train_ds = ZINC(root='../../data/zinc/', subset=True, split='train')
test_ds = ZINC(root='../../data/zinc/', subset=True, split='test')

In [5]:
len(train_ds)

10000

In [6]:
len(test_ds)

1000

In [7]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [8]:
model = GCNNet([21, 16, 16]).to(device)

In [9]:
for params in model.parameters():
    print(params.shape)

torch.Size([16])
torch.Size([16, 21])
torch.Size([16])
torch.Size([16, 16])
torch.Size([8, 16])
torch.Size([8])
torch.Size([1, 8])
torch.Size([1])


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)

In [11]:
loss = nn.MSELoss()

In [12]:
def train(model, train_loader, optimizer, loss):
    model.train()
    loss_acc = 0
    total_graphs = 0
    for graph_batch in train_loader:
        graph_batch = graph_batch.to(device)
        optimizer.zero_grad()
        
        x_oh = one_hot(graph_batch.x.flatten(), num_classes=21).type(torch.cuda.FloatTensor)
        preds = model(x_oh, graph_batch.edge_index, graph_batch.batch).squeeze()
        loss_val = loss(preds, graph_batch.y)
        loss_acc += loss_val.item()
        total_graphs += graph_batch.num_graphs
        loss_val.backward()
        optimizer.step()
        
    loss_acc /= total_graphs
    return loss_acc

In [66]:
from sklearn.metrics import r2_score

def validate(model, valid_loader, loss):
    model.eval()
    loss_acc = 0
    total_graphs = 0
    accuracy = []
    for graph_batch in valid_loader:
        graph_batch = graph_batch.to(device)
        x_oh = one_hot(graph_batch.x.flatten(), num_classes=21).type(torch.cuda.FloatTensor)
        preds = model(x_oh, graph_batch.edge_index, graph_batch.batch).squeeze()
        loss_val = loss(preds, graph_batch.y)
        loss_acc += loss_val.item()
        total_graphs += graph_batch.num_graphs
        
        with torch.no_grad():
            score = r2_score(graph_batch.y.cpu().numpy(), preds.cpu().numpy())
            score = torch.round(torch.tensor(score)*100)
            accuracy.append(score)
            model_accuracy = torch.mean(torch.tensor(accuracy))
            
        
    loss_acc /= total_graphs
    return loss_acc, model_accuracy

In [27]:
for batch in train_loader:
    batch1 = batch
    break

In [28]:
batch1.x.shape

torch.Size([773, 1])

In [29]:
batch1.batch.shape

torch.Size([773])

In [30]:
batch1.batch

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,
         7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8, 

In [68]:
train_loss = []
test_loss = []
valid_accuracy = []
for epoch in range(10):
    print('EPOCH:', epoch+1)
    print('Training...')
    loss_value = train(model, train_loader, optimizer, loss)
    train_loss.append(loss_value)
    print('Training Loss:', loss_value)

    print('Validating')
    loss_value, accuracy = validate(model, test_loader, loss)
    test_loss.append(loss_value)
    valid_accuracy.append(accuracy.item())
    print('Validation Loss:', loss_value)
    print('Accuracy:', accuracy.item())


EPOCH: 1
Training...
Training Loss: 0.08367350916862487
Validating
Validation Loss: 0.07814526653289795
Accuracy: 39.34375
EPOCH: 2
Training...
Training Loss: 0.07652328058481217
Validating
Validation Loss: 0.07186769837141037
Accuracy: 44.6875
EPOCH: 3
Training...
Training Loss: 0.07120568500757217
Validating
Validation Loss: 0.06729897040128707
Accuracy: 48.5625
EPOCH: 4
Training...
Training Loss: 0.0670227885901928
Validating
Validation Loss: 0.06337603205442428
Accuracy: 51.5625
EPOCH: 5
Training...
Training Loss: 0.06347308950424195
Validating
Validation Loss: 0.060301274418830875
Accuracy: 54.25
EPOCH: 6
Training...
Training Loss: 0.060303811007738115
Validating
Validation Loss: 0.058214160412549974
Accuracy: 56.4375
EPOCH: 7
Training...
Training Loss: 0.0576489982932806
Validating
Validation Loss: 0.05562109088897705
Accuracy: 58.40625
EPOCH: 8
Training...


KeyboardInterrupt: 